In [1]:
#util
import os
import time

#Math/arrays
import numpy as np
import pandas as pd

import optimizationFuncs as optim
import cpso
from importlib import reload

# Plotting
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d 

#File io
#from Bio.PDB import *
from loadFile import *
from makePDB import *

#PSO
from threeDMaxGrad import *
from myUtils import *
# Import PySwarms
import pyswarms as ps
import numpy.random as random

%load_ext autoreload
%autoreload 2

In [2]:
#==========================================================================
# Make directory if it doesn't exist
#--------------------------------------------------------------------------
if not os.path.exists('my_folder'):
    os.makedirs('my_folder')

In [3]:
## constants
#inFile = "../../examples/GSDB/inputs/KR_1mb/chr20_matrix.txt"
inFile = "../../examples/GSDB/inputs/KR_1mb/chr20_list.txt"
#inFile = "../../examples/input/chr19.txt"

## load file 
lstCons=loadFile.loadFileFunc(loadFile(), inFile)
 

this is a sparse matrix
Number of points :  1891
numBins  62.0
[[0.00000000e+00 1.00000000e+00 8.83912865e+04]
 [0.00000000e+00 5.50000000e+01 2.24361671e+03]
 [0.00000000e+00 2.50000000e+01 3.61152985e+03]
 ...
 [6.00000000e+01 6.20000000e+01 1.42017020e+04]
 [6.00000000e+01 6.10000000e+01 6.48508182e+04]
 [6.10000000e+01 6.20000000e+01 9.34392380e+04]]


In [4]:
lstCons[:,0] = lstCons[:,0].astype('int')
lstCons[:,1] = lstCons[:,1].astype('int')

scaleFactor = 25/max(lstCons[:,2])
lstCons[:,2] = lstCons[:,2]*scaleFactor
lstCons

array([[ 0.        ,  1.        , 16.61958593],
       [ 0.        , 55.        ,  0.42185132],
       [ 0.        , 25.        ,  0.67905031],
       ...,
       [60.        , 62.        ,  2.67024517],
       [60.        , 61.        , 12.19343883],
       [61.        , 62.        , 17.56871639]])

In [5]:
class outputObj:
    def __init__(self, xyzData, outputFile):
        self.xyzData = xyzData
        self.outputFile = outputFile
        
        #defaults
        self.recordName = []
        for i in range(len(xyzData)):
            self.recordName.append('ATOM')
            

    def myfunc(self):
        print("Hello my name is David")

In [6]:
def cost(xOyOz):
    cost = 0
    d = 0;
    structure = variables;
    

    #loop through existing data
    for k in range(len(lstCons)):
        print("yo2", lstCons)
        i = int(lstCons[k,0]);    j = int(lstCons[k,1]);    IF = lstCons[k,2];  dist = lstCons[k,3];
        
        if (IF <= 0) :
            continue
        

        if (abs(i - j) == 1 ) :
            IF = 1.0 * maxIF

        x1=structure[i][0];  x2=structure[j][0];
        y1=structure[i][1];  y2=structure[j][1];
        z1=structure[i][2];  z2=structure[j][2];

        str_dist = threeDMaxGrad.calEuclidianDist(threeDMaxGrad(), x1,y1,z1,x2,y2,z2 );     

        z = str_dist - dist;

        # objective function       
        d += (z**2);     

        print(d)

        # cost
        cost = -(n/2) - (n*np.log(np.sqrt(d/n)));

        return cost

In [7]:
def getSpear():
    SUM = 0.0;

    Len = int(n * (n - 1) / 2) 
    Dist = np.zeros(Len);
    WishDist = np.zeros(Len);
    count = 1;
    structure = variables;
    for k in range(len(lstCons)):
        i = int(lstCons[k,0]);    j = int(lstCons[k,1]);    IF = lstCons[k,2];  dist = lstCons[k,3];
        # structure distance   
        x1=structure[i][0];  x2=structure[j][0];
        y1=structure[i][1];  y2=structure[j][1];
        z1=structure[i][2];  z2=structure[j][2];

        str_dist = threeDMaxGrad.calEuclidianDist(threeDMaxGrad(), x1,y1,z1,x2,y2,z2 );
        SUM = SUM + ((str_dist - dist)**2);

        # calculate spearman_correlation and Pearson correlation
        if (i != j and IF > 0 and count<=Len  ):   
            Dist[count] = str_dist;
            WishDist[count]= dist;
            count = count + 1

    SUM = SUM / len(lstCons);    
    rmse = np.sqrt(SUM);

    # let's convert to a dataframe
    df = pd.DataFrame({'Dist': Dist, 'wishDist': WishDist})
    pearsonCoeff = df.corr(method = 'pearson')
    spearmanCoeff = df.corr(method = 'spearman')
    print("preoptim  : ", spearmanCoeff.iloc[0]['wishDist']) 

In [8]:
def opt_func(X):
    n_particles = X.shape[0]  # number of particles
    
    d = 0;
    #print(X)


    structure = variables #this is xyz
    allDists = np.zeros(n_particles)
    
    cost= 0
    for particle in range(n_particles):
        structure[:,changedXYZ] =  X[particle]
        
        #loop through existing data
        for k in range(len(lstCons)):
            i = int(lstCons[k,0]);    
            j = int(lstCons[k,1]);    
            IF = lstCons[k,2];  
            dist = lstCons[k,3];
            if (IF <= 0) :
                continue


            if (abs(i - j) == 1 ) :
                IF = 1.0 * maxIF

            x1=structure[i][0];  x2=structure[j][0];
            y1=structure[i][1];  y2=structure[j][1];
            z1=structure[i][2];  z2=structure[j][2];

            str_dist = threeDMaxGrad.calEuclidianDist(threeDMaxGrad(), x1,y1,z1,x2,y2,z2 );     

            z = str_dist - dist;

            # objective function  
            d += (z**2)
            
            #cost
            cost += -(n/2) - (n*np.log(np.sqrt(d/n)));
            
        
        allDists[particle] = cost
    
    return allDists

In [9]:
def opt_func_deltaXYZ(X):
    n_particles = X.shape[0]  # number of particles
    
    d = 0;
    #print(X)


    structure = variables #this is xyz
    allDists = np.zeros(n_particles)
    
    cost= 0
    for particle in range(n_particles):
        structure[changedN,:] =  X[particle]
        
        #loop through existing data
        for k in range(len(lstCons)):
            i = int(lstCons[k,0]);    
            j = int(lstCons[k,1]);    
            IF = lstCons[k,2];  
            dist = lstCons[k,3];
            if (IF <= 0) :
                continue


            if (abs(i - j) == 1 ) :
                IF = 1.0 * maxIF

            x1=structure[i][0];  x2=structure[j][0];
            y1=structure[i][1];  y2=structure[j][1];
            z1=structure[i][2];  z2=structure[j][2];

            str_dist = threeDMaxGrad.calEuclidianDist(threeDMaxGrad(), x1,y1,z1,x2,y2,z2 );     

            z = str_dist - dist;

            # objective function  
            d += (z**2)
            
            #cost
            cost += -(n/2) - (n*np.log(np.sqrt(d/n)));
            
        
        allDists[particle] = cost
    
    return allDists

In [11]:
## this is convert to distance
#CONVERT_FACTOR_R = np.arange(0.1, 2, .2) # this is alpha and can be looped
CONVERT_FACTOR_R=[0.9]
MAX_ITERATION = 500; # maximum number of iterations

AVG_DIST = 10.0  # an arbitrary distance

n=int(max(max(lstCons[:,0]),max(lstCons[:,1])))+1
print("n = ",n)

bestAlpha= CONVERT_FACTOR_R[0]
lstConsReset = lstCons
for CONVERT_FACTOR in CONVERT_FACTOR_R : 
    lstCons = lstConsReset
    ## Find the average IF
    avgIF = 0.0
    for i in range(len(lstCons)):
        avgIF = avgIF + float(lstCons[i][2])
    avgIF = avgIF/len(lstCons)
    
    maxIF = 0.0
    ## scale average distance to AVG_DIST
    avgDist = 10.0;
    avgAdjIF = 0.0;
    avgAdjCount = 0;
    totalIF = 0;

    for i in range(len(lstCons)):
        x = lstCons[i][0]
        y = lstCons[i][1]
        IF = lstCons[i][2]
        lstCons[i][2] = IF / avgIF  # normallize IF by avgIF
        IF = lstCons[i][2]
        dist = 1/(IF**CONVERT_FACTOR)
        avgDist = avgDist + dist

        totalIF = totalIF +  IF

        if ( IF > maxIF):
            maxIF = lstCons[i][2]
        # Find the adjacent position IF
        if ( abs(x-y)==1):
            avgAdjCount= avgAdjCount+1
            avgAdjIF =  avgAdjIF + IF

    avgDist = avgDist/len(lstCons)
    avgAdjIF = avgAdjIF/avgAdjCount

    maxIF = min(avgAdjIF, maxIF)

    ## TODO Add adjacent if none exist
    
    print('TODO Added missing adjacent constraint...')

    print('Number of constraints: = ', n)
    maxD = 0
    distResultsList= []
    for i in range(len(lstCons)):
        IF = lstCons[i,2];
        dist = AVG_DIST/ ((IF**CONVERT_FACTOR)* avgDist)
        distResultsList.append(dist)
        if (dist > maxD):
            maxD = dist;

    result = np.hstack((lstCons, np.atleast_2d(distResultsList).T))
    lstCons = result
    print('Max distance is: = \n', maxD); 
    
    ## Optimization
    ## Initialize Structure
    #=========================================================================


    thisStr =  [];
    R = [-0.5,0.5];
    for i in range(n):
        xyz = np.array([random.random(),random.random(),random.random()]) * (R[1]-R[0]) + min(R) 
        thisStr.append(xyz)
    
    ## Variables declaration
    #=========================================================================
    thislen = len(lstCons)

    variables = thisStr;
    
    ######### PSO
    variables = np.array(variables)

    dist = 1/(IF**CONVERT_FACTOR)


    
    cpso = reload(cpso)
    dims = 3
    llim = np.repeat (-20, dims)
    rlim = np.repeat (20, dims)
    
    getSpear()
    
    # Perform optimization
    for i in range(n):
        changedXYZ = np.random.randint(3)
        changedN = np.random.randint(n)
        # Returns the swarm --> Also internally creates a chaotic generator object
        hecu = cpso.HECS_PSO(opt_func_deltaXYZ, llim, rlim, 10)
        #hecu = cpso.HECS_PSO(opt_func, llim, rlim, 10)
        a, b, opt = hecu.optimize(max_iters = 10, print_iters=True)
        #print(opt)
        print(a)
        #print(b)
        #variables[:,changedXYZ] = a      
        variables[changedN,:] = a  
        
        getSpear()

        
    

    #========================================================================
    # scoring using spearman correlation, pearson correlation and  RMSD     
    #------------------------------------------------------------------------
    # calculate rmse    
    SUM = 0.0;

    Len = int(n * (n - 1) / 2) 
    Dist = np.zeros(Len);
    WishDist = np.zeros(Len);
    count = 1;
    structure = variables;
    for k in range(len(lstCons)):
        i = int(lstCons[k,0]);    j = int(lstCons[k,1]);    IF = lstCons[k,2];  dist = lstCons[k,3];
        # structure distance   
        x1=structure[i][0];  x2=structure[j][0];
        y1=structure[i][1];  y2=structure[j][1];
        z1=structure[i][2];  z2=structure[j][2];

        str_dist = threeDMaxGrad.calEuclidianDist(threeDMaxGrad(), x1,y1,z1,x2,y2,z2 );
        SUM = SUM + ((str_dist - dist)**2);

        # calculate spearman_correlation and Pearson correlation
        if (i != j and IF > 0 and count<=Len  ):   
            Dist[count] = str_dist;
            WishDist[count]= dist;
            count = count + 1

    SUM = SUM / len(lstCons);    
    rmse = np.sqrt(SUM);

    # let's convert to a dataframe
    df = pd.DataFrame({'Dist': Dist, 'wishDist': WishDist})
    pearsonCoeff = df.corr(method = 'pearson')
    spearmanCoeff = df.corr(method = 'spearman')
   
    if CONVERT_FACTOR == CONVERT_FACTOR_R[0]:#first run
        bestSpearmanRHO = spearmanCoeff.iloc[0]['wishDist']
        bestPearsonRHO = pearsonCoeff.iloc[0]['wishDist']
        bestMat =  variables
    elif bestSpearmanRHO < spearmanCoeff.iloc[0]['wishDist']:
        bestSpearmanRHO = spearmanCoeff.iloc[0]['wishDist']
        bestPearsonRHO = pearsonCoeff.iloc[0]['wishDist']
        bestMat =  variables
        bestAlpha = CONVERT_FACTOR

print("RMSE  : ", rmse)    
print("bestPearsonRHO  : ", bestPearsonRHO) 
print("bestSpearmanRHO  : ", bestSpearmanRHO) 
print("Best Alpha : ",bestAlpha)

n =  63
TODO Added missing adjacent constraint...
Number of constraints: =  63
Max distance is: = 
 66.69620869552902
preoptim  :  0.1744301811214625
10
[18.51540592 -7.6542474  19.98262656]
preoptim  :  0.16038576343503796
10
[  7.00018965 -19.99992622  13.11232219]
preoptim  :  0.14285695387174713
10
[ 16.45855632 -15.31020534  19.96263262]
preoptim  :  0.13703043997436767
10
[18.15263599 15.44165628 -3.29691705]
preoptim  :  0.14198404275914966
10
[  0.90345091  19.21697287 -19.96595237]
preoptim  :  0.1472993090364762
10
[ 11.27629845  12.46935756 -19.92936997]
preoptim  :  0.14312279992714877
10
[ 16.71352382  13.33294032 -12.82262285]
preoptim  :  0.15100096391068818
10
[ 11.57984501 -19.26477376 -11.72400548]
preoptim  :  0.13502401816775308
10
[ 19.7840125   13.56146508 -17.0917237 ]
preoptim  :  0.1393947531618586
10
[  3.2989226  -17.62030446  18.87273978]
preoptim  :  0.14829450037559178
4

KeyboardInterrupt: 

In [ ]:
#========================================================================
# create pdb
#------------------------------------------------------------------------
#Increase structure Size

xyz4pdb = myUtils.convert2xyz(myUtils(), n, bestMat) 
scale=100/np.amax(xyz4pdb)
xyz4pdb = xyz4pdb* scale
#output pdb file
outputData= np.round_(xyz4pdb ,3 )

outputFile = 'yolo.pdb' #output directory.


if os.path.exists(outputFile):
    os.remove(outputFile)

output = outputObj(outputData,outputFile)

pdbMaker = makePDB()
makePDB.mat2pdb(pdbMaker, output) # Converts the mat XYZ coordinates to PDB format.

In [ ]:
opt